This notebook contains a workflo to adding contrast and correlation from GLCM texture features

References:
https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycoprops

https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_glcm.html

https://prism.ucalgary.ca/bitstream/handle/1880/51900/texture%20tutorial%20v%203_0%20180206.pdf?sequence=11&isAllowed=y

https://stackoverflow.com/questions/50834170/image-texture-with-skimage

https://www.tandfonline.com/doi/epdf/10.1080/01431161.2016.1278314?needAccess=true&role=button

In [1]:
import os
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

import sample_rasters as sr
from rasterio.crs import CRS
from rasterio.transform import rowcol

from shapely.geometry import box

import planetary_computer as pc

from skimage.feature import graycomatrix, graycoprops

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

# csv with the points for which to add spectral window features
csv_name = 'spectral_window_model3070FP_train_2020.csv'
#fp = '/home/jovyan/msai4earth-esa/iceplant_detection/processing_results/model_3070/model3070_test_2020.csv'
fp = os.path.join(os.getcwd(),'temp',csv_name)
all_pts = pd.read_csv(fp)

window_r = 5  #in pixels
side = 2*window_r + 1

distances = [1]
angles = [0, np.pi/2]
props = ['contrast', 'correlation']

# ***************************************************
# ***************************************************

In [3]:
itemids = pd.read_csv(sr.path_to_aoi_itemids_csv())

# temporary folder for aux rasters
folp = os.path.join(os.getcwd(),'temp','aux_naip_rasters')
if os.path.exists(folp) == False:
    os.mkdir(folp)

In [4]:
t0 = time.time() # initial time tracker

sampled_pts = [] # sampled pts from each scene are collected here

N = len(itemids)  # counter to finish

crss = all_pts.pts_crs.unique()

for i in range(len(itemids)):
    # ---------------------------------------
    # open raster reader for NAIP scene
    itemid = itemids.itemid[i]
    item = sr.get_item_from_id(itemid)    
    href = pc.sign(item.assets["image"].href)
    naip_rast_r = rioxr.open_rasterio(href) 

    all_pts_scene = all_pts.loc[all_pts['naip_id'] == itemid]
    if len(all_pts_scene) !=0:
        for crs_str in crss:
            pts_scene = all_pts_scene[all_pts_scene.pts_crs == crs_str]  
    
            if len(pts_scene) !=0:
                crs = CRS.from_string(crs_str)
                pts_scene_df = sr.geodataframe_from_csv(df = pts_scene, lon_label='x', lat_label='y', crs=crs)
                pts_col = pts_scene_df.to_crs(naip_rast_r.rio.crs).geometry

                samples = []
                for pt in pts_col:
                    # make a small window out of raster, all bands
                    row, col = rowcol(naip_rast_r.rio.transform(), pt.x, pt.y)
                    windows = naip_rast_r[:, row-window_r:row+window_r+1, col-window_r:col+window_r+1]
                   
                    # calculate GLCM on that small window, all bands
                    # calculate the contrast and correlation on that window (upwards direction)
                    # add those as features for that pixel 
                    
                    pixel_feats = []
                    for angle in angles:
                        for band in range(4):
                            glcm = graycomatrix(windows[band],
                                        distances=distances,
                                        angles=[angle])
                            pixel_feats.append(graycoprops(glcm, 'contrast')[0,0])
                            pixel_feats.append(graycoprops(glcm, 'correlation')[0,0])
                    samples.append(pixel_feats)
                    
                # ---------------------------------------
                # Add all derived spectral data to pts dataframe
                new_features = pd.DataFrame(np.vstack(samples), 
                                            columns = ['r_contE', 'r_corrE', 'g_contE', 'g_corrE', 'b_contE', 'b_corrE', 'nir_contE', 'nir_corrE',
                                                       'r_contN', 'r_corrN', 'g_contN', 'g_corrN', 'b_contN', 'b_corrN', 'nir_contN', 'nir_corrN'])   
                pts = pd.concat([pts_scene, new_features.set_index(pts_col.index)], axis=1)                

                # -----------------------------
                # collect all points from each polygon in the scene
                sampled_pts.append(pts)                    

    # ---------------------------------------
    # processing message
    N = N-1                
    print('REMAINING: ', N, 'scenes', end="\r")

print('FINISHED PROCESSING')       
     
# ---------------------------------------
# create data frame with all points
sampled_pts = pd.concat(sampled_pts).sort_index()                

FINISHED PROCESSINGss


In [5]:
sampled_pts

,x,y,pts_crs,aoi,naip_id,r,r_max,r_min,r_diff,r_avg,...,nir_contE,nir_corrE,r_contN,r_corrN,g_contN,g_corrN,b_contN,b_corrN,nir_contN,nir_corrN
0,240941.232268,3.812115e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,69,75,69,6,71.777780,...,3.936364,0.440369,9.572727,0.491427,7.390909,0.548292,1.363636,0.458270,2.854545,0.632150
1,237666.773215,3.812734e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,78,80,68,12,74.333336,...,186.400000,0.795774,91.409091,0.861490,49.100000,0.810525,43.572727,0.915919,259.700000,0.709684
2,239080.498643,3.811790e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,96,99,87,12,93.444440,...,48.027273,0.820316,36.209091,0.795060,14.390909,0.844345,11.663636,0.940831,21.263636,0.915150
3,238496.925128,3.810764e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,93,93,87,6,90.222220,...,12.000000,0.750304,108.218182,0.729098,74.272727,0.689768,80.481818,0.816136,20.236364,0.569546
4,241568.186396,3.811966e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,72,83,67,16,75.333336,...,8.172727,0.876273,22.618182,0.911081,8.863636,0.801254,2.781818,0.828384,10.981818,0.851220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,-120.500582,3.449420e+01,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,114,181,104,77,129.444440,...,150.090909,0.902062,459.563636,0.883972,431.963636,0.887571,343.590909,0.894314,143.845455,0.904856
1789,-120.469562,3.447049e+01,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,102,114,93,21,104.222220,...,20.009091,0.781880,57.163636,0.615369,48.636364,0.680252,33.627273,0.727875,33.363636,0.641011
1790,734086.618872,3.815760e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,54,74,45,29,58.888890,...,310.518182,0.869727,523.190909,0.846267,396.463636,0.824812,202.745455,0.889334,691.400000,0.699743
1791,-120.438094,3.445419e+01,EPSG:4326,point_conception,ca_m_3412037_nw_10_060_20200607,90,108,87,21,95.666664,...,73.445455,0.568456,99.363636,0.857715,52.854545,0.866812,29.718182,0.896124,29.054545,0.877613


In [6]:
sampled_pts.columns

Index(['x', 'y', 'pts_crs', 'aoi', 'naip_id', 'r', 'r_max', 'r_min', 'r_diff',
       'r_avg', 'r_entr', 'g', 'g_max', 'g_min', 'g_diff', 'g_avg', 'g_entr',
       'b', 'b_max', 'b_min', 'b_diff', 'b_avg', 'b_entr', 'nir', 'nir_max',
       'nir_min', 'nir_diff', 'nir_avg', 'nir_entr', 'ndvi', 'ndvi_max',
       'ndvi_min', 'ndvi_diff', 'ndvi_avg', 'ndvi_entr', 'year', 'month',
       'day_in_year', 'r_entr5', 'g_entr5', 'b_entr5', 'nir_entr5',
       'ndvi_entr5', 'iceplant', 'geometry', 'r_contE', 'r_corrE', 'g_contE',
       'g_corrE', 'b_contE', 'b_corrE', 'nir_contE', 'nir_corrE', 'r_contN',
       'r_corrN', 'g_contN', 'g_corrN', 'b_contN', 'b_corrN', 'nir_contN',
       'nir_corrN'],
      dtype='object')

In [7]:
sampled_pts = sampled_pts[['x', 'y', 'pts_crs', #  point location
             'aoi', 'naip_id', #'polygon_id',  # sampling info
             'r', 'r_max', 'r_min', 'r_diff', 'r_avg', 'r_entr', # spectral
             'g', 'g_max', 'g_min', 'g_diff', 'g_avg', 'g_entr',
             'b', 'b_max', 'b_min', 'b_diff', 'b_avg', 'b_entr',
             'nir', 'nir_max', 'nir_min', 'nir_diff', 'nir_avg', 'nir_entr',
             'ndvi', 'ndvi_max', 'ndvi_min', 'ndvi_diff', 'ndvi_avg', 'ndvi_entr',   
             'year', 'month', 'day_in_year', # date
             'r_contE', 'r_corrE', 'g_contE', 'g_corrE', 'b_contE', 'b_corrE', 'nir_contE', 'nir_corrE',
             'r_contN', 'r_corrN', 'g_contN', 'g_corrN', 'b_contN', 'b_corrN', 'nir_contN', 'nir_corrN',                           
            'r_entr5','g_entr5', 'b_entr5', 'nir_entr5', 'ndvi_entr5',                          
             'iceplant'
             ]] 


In [8]:
fp = os.path.join(os.getcwd(),'temp', 'glcm_'+csv_name)
sampled_pts.to_csv(fp, index=False)